In [1]:
from abc import ABC, abstractmethod
from Constructor.objectTypes import Value, DataFrame
from Constructor.abstractNode import BaseNode
from Constructor.knotslinkers import BaseInputKnot, BaseOutputKnot

In [7]:
class ImportDataNode(BaseNode):
    priority = 0
    def __init__(self):
        super().__init__(priority=self.priority)
        self.AllInputs = []
        self.AllOutputs = [{'position': 1, 'knot': BaseOutputKnot(DataFrame())}]

        self._inside = None
        self._mutable = False

    def change_condition(self, **param):
        from pandas import read_csv
        if not 'filepath' in param:
            raise KeyError('No filepath')
        else:
            filepath = param['filepath']
        if not 'sep' in param:
            sep = ','
        else:
            sep = param['sep']
        if not 'header' in param:
            header = 0
        else:
            header = param['header']
        if not 'index_col' in param:
            index_col = 0
        else:
            index_col = param['index_col']
            
        buffer = self._inside
        self._inside = read_csv(filepath_or_buffer=filepath, sep=sep, header=header, index_col=index_col)

        if buffer != self._inside:
            self._mutable = True
        del buffer

    def execute_node(self):
        if self._mutable:
            self.AllOutputs[0]['knot'].changed_condition(self._inside)
        self._mutable = False

    def NodeIndex(self):
        return 'D_001'

class GetABSValue(BaseNode):
    priority = 3
    def __init__(self):
        super().__init__(priority=self.priority)
        self.AllInputs = [{'position': 1, 'knot': BaseInputKnot(Value())}, {'position': 2, 'knot': BaseInputKnot(Value())}]
        self.AllOutputs = []

    def convert(self):
        return abs(self.AllInputs[0]['knot'].KnotObject.receive())

    def show_condition(self):
        for inputs in self.AllInputs:
            print(f"{inputs['position']}, condition: {inputs['knot'].KnotObject.receive()}")

    def NodeIndex(self):
        return 'G_001'


class ConstNode(BaseNode):
    priority = 3
    def __init__(self):
        super().__init__(priority=self.priority)
        self.AllInputs = list()
        self.AllOutputs = [{'position': 1, 'knot': BaseOutputKnot(Value())}]

        self._inside = None
        self._mutable = None

    def change_condition(self, value):
        buffer = self._inside
        self._inside = value

        if buffer != self._inside:
            self._mutable = True
        del buffer

    def execute_node(self):
        if self._mutable:
            self.AllOutputs[0]['knot'].changed_condition(self._inside)
        self._mutable = False

    def NodeIndex(self):
        return 'G_002'

getValue = GetABSValue()
CONST = ConstNode()
getValue.create_parent_link(self_inp_position=0,parentNode=CONST, parent_out_position=0)

for i in [1,1,2,3]:
   CONST.change_condition(i)
   CONST.execute_node()

getValue.show_condition()

EXECUTE
1
EXECUTE
2
EXECUTE
3
1, condition: 3
2, condition: None
